This notebook focuses on the first 2 days of the simulation. Where the hadley circulation appears to weaken rapidly

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import numpy as np
import xarray as xr

from src.data import SAMRun
from uwnet import thermo

In [ ]:
class NGAqua:
    @property
    def data_2d(self):
        path = "../data/raw/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc"
        ds = xr.open_dataset(path).sortby('time')
        return ds.assign(NPNN=thermo.net_precipitation_from_prec_evap(ds))


low_res = "../data/runs/2019-05-01-model268-epoch5-res128x64x34/"



ng = NGAqua()
lo_run = SAMRun(low_res, case='control')

# this is expensive. so preloads
data_2d = ng.data_2d

# Snapshots

In [ ]:
def plot_pw(run, key='PW', time=101):
    run.data_2d[key].sel(time=time, method='nearest').plot()

## PW

In [ ]:
plot_pw(ng, 'PW', 102)

In [ ]:
plot_pw(lo_run, 'PW', 102)

In [ ]:
data_2d['y'] = lo_run.data_2d.y
data_2d['x'] = lo_run.data_2d.x

In [ ]:
diff = lo_run.data_2d.PW - data_2d.PW

In [ ]:
time = 101.375
diff.sel(time=time).plot()
plt.title(f"PW SAM - NG-Aqua (mm) at day {time}");

# Water Budget 

In [ ]:
def water_budget(data_2d):
    storage = data_2d.PW.differentiate('time')
    advection = storage + data_2d.NPNN
    
    return xr.Dataset({'storage': storage, 'advection': advection, 'net_precip':  data_2d.NPNN})


In [ ]:
ng_h20 = water_budget(data_2d.sel(time=slice(100, 110)))
lo_h20 = water_budget(lo_run.data_2d.sel(time=slice(100, 110)))

In [ ]:
def plot_water_budget(snapshot, M=100):
    if M is None:
        m = None
    else:
        m = -M
    
    fig, (a,b, c) = plt.subplots(1, 3, figsize=(13, 3))
    snapshot.advection.plot(vmax=M, vmin=m, cmap='RdBu_r', ax=b)
    snapshot.storage.plot(vmax=M, vmin=m, cmap='RdBu_r', ax=a)
    snapshot.net_precip.plot(vmax=M, vmin=m, cmap='RdBu_r', ax=c)
    plt.tight_layout()
    
def plot_water_budget_zonal_mean(mean, axs=None):
    if axs is None:
        fig, axs = plt.subplots(1, 3, figsize=(13, 3))
        
    a, b, c = axs
        
    mean.advection.plot(ax=b)
    mean.storage.plot(ax=a)
    mean.net_precip.plot(ax=c)
    
    if axs is None:
        plt.tight_layout()
        
    return axs

def compare_zonal_mean(field, time):
    a = data_2d[field].sel(time=time).mean('x')
    b = lo_run.data_2d[field].sel(time=time).mean('x')
    
    a.plot()
    b.plot()
    plt.legend(['NG-Aqua', '160km'])

In [ ]:
plot_water_budget(lo_h20.sel(time=101))

In [ ]:
plot_water_budget(ng_h20.sel(time=101))

## zonal mean

In [ ]:
axs=  plot_water_budget_zonal_mean(ng_h20.sel(time=101.0).mean('x'))
plot_water_budget_zonal_mean(lo_h20.sel(time=101.0).mean('x'), axs=axs);
plt.legend(['NG', '160km'])

Looks like there is more advective drying of the subtropics, and not enough evaporation (negative net precip) there.

In [ ]:
compare_zonal_mean('W500', 102)

### Velocity

In [ ]:
with plt.rc_context({'axes.prop_cycle':  plt.cycler(linestyle=['-','--'])  * plt.cycler(color=['b', 'g']) }):
    compare_zonal_mean('VSFC', 102)
    compare_zonal_mean('V850', 102)

In [ ]:
with plt.rc_context({'axes.prop_cycle':  plt.cycler(linestyle=['-','--'])  * plt.cycler(color=['b', 'g']) }):
    compare_zonal_mean('USFC', 102)
    compare_zonal_mean('U850', 102)

Large velocity biases develop in the first few days.

# accumulated water budget

the budget is accumulated in time to produce less noisy results.

In [ ]:
def accumulate_budget(water_budget, t):
    time = water_budget.time
    dt = time[1] - time[0]
    return water_budget.sel(time=slice(100, t)).sum('time') * dt

lo_cumulative_budget = accumulate_budget(lo_h20, time)
ng_cumulative_budget = accumulate_budget(ng_h20, time)
plot_water_budget(lo_cumulative_budget, M=20)

In [ ]:
plot_water_budget(ng_cumulative_budget, M=20)

In [ ]:
run = SAMRun("../data/runs/2019-03-14-model268-epoch5-debiased/", case='control')
debiased_water = water_budget(run.data_2d)
db_cum_budget = accumulate_budget(debiased_water, time)

In [ ]:
axs = plot_water_budget_zonal_mean(ng_cumulative_budget.mean('x'));
plot_water_budget_zonal_mean(lo_cumulative_budget.mean('x'), axs=axs);
plot_water_budget_zonal_mean(db_cum_budget.mean('x'), axs=axs)
plt.legend(['NG-Aqua', 'NN', 'NN-Debiased'])

# Compare with the debiased run

It is not clear if the errors in the initial tendency are because of the NN's bias. To examine this, we should look at the debiased run.

In [ ]:
plot_water_budget(db_cum_budget, M=20)

In [ ]:
diff = run.data_2d.PW - data_2d.PW

In [ ]:
time = 101.5
diff.sel(time=time).plot()
plt.title(f"PW SAM - NG-Aqua (mm) at day {time}");

In [ ]:
def plot_tropics_mean_pw(case, key):
    z= case.data_2d[key].sel(y=slice(4.5e6, 5.5e6), time=slice(None, 110)).mean(['x', 'y'])
    z.plot()

    
def plot_tropics_avg_cases(cases, labels, key='PW'):
    for case in cases:
        plot_tropics_mean_pw(case, key)
    plt.legend(labels)
    plt.title(f"Tropical average of {key}")

    

In [ ]:
plot_tropics_avg_cases([ng, lo_run, run], ['NG-Aqua', 'NN', 'NN-Debiased'])

Why is there a discrepency between NG-Aqua and the NN runs in the initial time step?

In [ ]:
plot_tropics_avg_cases([ng, lo_run, run], ['NG-Aqua', 'NN', 'NN-Debiased'], key='NPNN')

The debiased network does produce more rainfall in the initial times.

In [ ]:
plot_tropics_avg_cases([ng, lo_run, run], ['NG-Aqua', 'NN', 'NN-Debiased'], key='W500')

The average vertical velocity is also a little better in the initial time period, although there it does seem abnormally small in the beginning. Is this due to a mistreatment of the diurnal cycle, which was an issue fixed by this [commit](https://github.com/nbren12/uwnet/commit/d860c42abf7d777e74ed21c1529b1934c42a24fe)? Let's look at the value of SOLIN

In [ ]:
run.open_debug(60)['SOLIN'].mean()

Indeed this is negative, which should not be the case. This could explain the inadequate vertical velocity at the initial time. I will need to re-run this simulation.

# Re-run debiased simulation for first 2 days

In [ ]:
run_path = "data/runs/2019-05-14-model268-epoch5-debiased-res128x64x34"

Make the run directory and execute the run with the following lines:

In [ ]:
%%script bash --proc proc --bg --err err --out out

# exit when any command fails
set -e

cd ../

run_path="data/runs/2019-05-14-model268-epoch5-debiased-res128x64x34"

if [[ -d "$run_path" ]]
then
echo "Output directory already exists"
exit 1
fi

python -m src.sam.create_case \
     -p assets/parameters_sam_neural_network_2day.json  \
     -s /home/disk/eos4/nbren12/work/uwnet/ext/sam/OBJ/128x64x34 \
     -nn models/268/5.debiased.pkl \
     -n data/raw/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX \
     -r ext/sam/RUNDATA \
     $run_path


cd $run_path 

execute_run.sh . > out 2> err

In [ ]:
# execute to get the current time-step of the model
!grep NSTEP ../{run_path}/out | tail -n 1

# Is process done? 
proc.poll()

In [ ]:
# convert the 2D output

!cd ../{run_path}/OUT_2D && 2Dbin2nc *.2Dbin > /dev/null

In [ ]:
run_db_fixed = SAMRun(f"../{run_path}", case='control')
plot_tropics_avg_cases([ng, lo_run, run_db_fixed], ['NG-Aqua', 'NN', 'NN-Debiased'], key='NPNN')
plt.xlim(right=102)

In [ ]:
plot_tropics_avg_cases([ng, lo_run, run_db_fixed], ['NG-Aqua', 'NN', 'NN-Debiased'], key='W500')
plt.xlim(right=102)

Overall this simulation looks pretty similar to the one with the incorrect SOLIN.